# Sentiment analysis

In [79]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification

# import json

# # Load the data from JSON file
# with open("sentiment_analysis.json", "r") as f:
#     data = json.load(f)

# print(data.items)

# # Define the training data
# train_data = []
# sub_labels = {}
# for top_level_label, sub_level_labels in data.items():
#     for sub_level_label_data in sub_level_labels:
#         text = sub_level_label_data["text"]
#         sub_level_label = sub_level_label_data["sub_level_label"]
#         train_data.append((text, top_level_label, sub_level_label))
#         if top_level_label not in sub_labels:
#             sub_labels[top_level_label] = []
#         sub_labels[top_level_label].append(sub_level_label)



# # Define the mapping between top-level labels and integers
# top_level_label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# # Define the mapping between sub-level labels and integers
# sub_level_label_map = {sub_label: i for i, sub_label in enumerate(set([sub_label for sub_labels_list in sub_labels.values() for sub_label in sub_labels_list]))}

# # Convert the training data labels to integers using the label_map and sub_label_map
# # A tensor is a multi-dimensional array that looks like a numpy array, it's used for neural networks
# top_level_labels = torch.tensor([top_level_label_map[data[1]] for data in train_data])
# sub_level_labels = torch.tensor([sub_level_label_map[sub_label] for data in train_data for sub_label in sub_labels[data[1]]])

# # Load the pre-trained BERT model and tokenizer
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(top_level_label_map))
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# # Tokenize the training data and convert to tensors
# inputs = tokenizer.batch_encode_plus([data[0] for data in train_data], padding=True, truncation=True, return_tensors="pt")

# # Fine-tune the model on the training data
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
# loss_fn = torch.nn.CrossEntropyLoss()
# for epoch in range(10):
#     optimizer.zero_grad()
#     outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=top_level_labels)
#     loss = outputs.loss
#     loss.backward()
#     optimizer.step()
    

#     print(f"Epoch {epoch+1}, Loss: {loss.item()}")

#     # Evaluate the model on the training data
#     predictions = outputs.logits.argmax(axis=1)
#     accuracy = (predictions == top_level_labels).sum()


In [132]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import json
import nltk

# Load the data from JSON file
with open("sentiment_analysis_v2.json", "r") as f:
    data = json.load(f)

# Define the training data
train_data = []
for datum in data:
    texts = datum["text"]
    label = datum["intent"]
    for text in texts:
        train_data.append((text, label))

# Define the mapping between top-level labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
labels = torch.tensor([label_map[data[1]] for data in train_data])

# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the training data and convert to tensors
inputs = tokenizer.batch_encode_plus([data[0] for data in train_data], padding=True, truncation=True, return_tensors="pt")

# Fine-tune the model on the training data
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4) #default 5e-5
loss_fn = torch.nn.CrossEntropyLoss()
for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Evaluate the model on the training data
    predictions = outputs.logits.argmax(axis=1)
    accuracy = (predictions == labels).sum()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1, Loss: 3.8207039833068848
Epoch 2, Loss: 3.812309980392456
Epoch 3, Loss: 3.723907232284546
Epoch 4, Loss: 3.639765739440918
Epoch 5, Loss: 3.2687788009643555
Epoch 6, Loss: 3.286609649658203
Epoch 7, Loss: 2.847144842147827
Epoch 8, Loss: 2.4352078437805176
Epoch 9, Loss: 2.1434130668640137
Epoch 10, Loss: 1.8343788385391235
Epoch 11, Loss: 1.512740969657898
Epoch 12, Loss: 1.3305459022521973
Epoch 13, Loss: 1.3323346376419067
Epoch 14, Loss: 0.9725267887115479
Epoch 15, Loss: 0.8231538534164429
Epoch 16, Loss: 0.694115400314331
Epoch 17, Loss: 0.542227566242218
Epoch 18, Loss: 0.4735408127307892
Epoch 19, Loss: 0.3863665461540222
Epoch 20, Loss: 0.3214249610900879


In [133]:
# Sample test question
test_question = "How does it work with refunds?"
def get_intent(question):

    # Tokenize the test question and convert to tensors
    inputs = tokenizer.encode_plus(question, padding=True, truncation=True, return_tensors="pt")

    # Get the model's prediction for the test question
    with torch.no_grad():
        outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    predictions = outputs.logits.argmax(axis=1)
    predicted_label = list(label_map.keys())[list(label_map.values()).index(predictions[0].item())]
    print(f"Predicted intent: {predicted_label}")
    return predicted_label

get_intent(test_question)



Predicted intent: refunds_legal_statement


'refunds_legal_statement'

## Sentiment analysis query function

In [82]:
# Test the model

# def predict_intent(text):
#     top_level_inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt")
#     top_level_outputs = model(top_level_inputs["input_ids"], attention_mask=top_level_inputs["attention_mask"])
#     # top_level_predicted_labels = torch.argsort(top_level_outputs.logits, descending=True).tolist()[0]
#     top_level_predicted_label = torch.argmax(top_level_outputs.logits).item()
#     top_level_predicted_intent = [k for k, v in top_level_label_map.items() if v == top_level_predicted_label]
    
#     sub_level_inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt")
#     sub_level_outputs = model(sub_level_inputs["input_ids"], attention_mask=sub_level_inputs["attention_mask"])
#     sub_level_predicted_label = torch.argmax(sub_level_outputs.logits).item()
#     # sub_level_predicted_intents = [k for k, v in sub_level_label_map.items() if v in sub_level_predicted_labels]
#     sub_level_predicted_intent = [k for k, v in sub_level_label_map.items() if v == sub_level_predicted_label]
    
#     return top_level_predicted_intent[0] if top_level_predicted_intent else None, sub_level_predicted_intent[0] if sub_level_predicted_intent else None



# top_level_intents, sub_level_intents = predict_intent("How can i change my password")
# print(top_level_intents, sub_level_intents)


external_platform_settings rights


## GPT

In [84]:
# GPT model herel 
%load_ext autoreload
%autoreload 2

from gpt import GPT

gpt_model = GPT()

#gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')


d:\school\iYYU\gpt.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  serie = serie.str.replace('\\n', ' ')


In [137]:
import gradio as gr
import time
import random

def get_object_by_intent(intent):
    for object in data:
        if object['intent'] == intent:
            return object
    return None

theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

with gr.Blocks(theme=theme, css="chat/chat.css") as demo:
    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False, tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent = get_intent(question=user_message)
        print('intent:', intent)
        
        intent =  get_object_by_intent(intent)
        
        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent['use_gpt']

        # if the intent is 'privacy_policy' or 'legal_statement', use GPT to generate a response
        response = '(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message)) if use_gpt else '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0])


        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        time.sleep(1)
        return history

    submit.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit.click(lambda: None, None, chatbot, queue=False)

demo.launch()



Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


Predicted intent: location_password_change
intent: location_password_change
Predicted intent: privacy_policy_updates
intent: privacy_policy_updates
